# Data Extraction and Training with DEGIS Package

This notebook demonstrates how to use the DEGIS package to:
1. Extract CLIP embeddings from images
2. Generate color histograms and edge maps
3. Train color disentanglement models

Based on the logic from `main.py` but using the new package structure.


## 1. Setup and Imports


In [1]:
# Install the package in development mode if needed
# !pip install -e .

import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from multiprocessing import cpu_count
import os
import time
import platform
import psutil
import shutil

# Import the DEGIS package
import degis
from degis.data.dataset import UnifiedImageDataset

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model projection_dim: 1280


## 2. System Profiling


In [2]:
def print_system_profile():
    print("=== SYSTEM PROFILE ===")
    print("Python:", platform.python_version())
    print("PyTorch:", torch.__version__)
    print("CPU cores:", psutil.cpu_count(logical=True))
    vm = psutil.virtual_memory()
    print(f"RAM: {vm.total/1e9:.1f} GB, free {vm.available/1e9:.1f} GB")
    
    # Check if /data exists, otherwise check current directory
    if os.path.exists("/data"):
        du = shutil.disk_usage("/data")
        print(f"/data disk: total {du.total/1e9:.1f} GB, free {du.free/1e9:.1f} GB")
    else:
        du = shutil.disk_usage(".")
        print(f"Current disk: total {du.total/1e9:.1f} GB, free {du.free/1e9:.1f} GB")
    
    print("CUDA available:", torch.cuda.is_available())
    if torch.cuda.is_available():
        i = torch.cuda.current_device()
        print("GPU:", torch.cuda.get_device_name(i))
        print(f"VRAM total: {torch.cuda.get_device_properties(i).total_memory/1e9:.1f} GB")
    print("======================")

print_system_profile()


=== SYSTEM PROFILE ===
Python: 3.12.3
PyTorch: 2.8.0+cu128
CPU cores: 256
RAM: 540.8 GB, free 456.8 GB
/data disk: total 1099.5 GB, free 303.5 GB
CUDA available: True
GPU: NVIDIA GeForce RTX 5090
VRAM total: 33.7 GB


## 3. Load Dataset and Create Data Loader


In [3]:
csv_path = "/data/thesis/adimagenet_manifest.csv"
batch_size = 512
embeddings_path = "/data/thesis/models/adimagenet_embeddings.npy"
colour_hist_path = "/data/thesis/data/adimagenet_color_histograms_hcl_514.npy" 
edge_maps_path = "/data/thesis/data/adimagenet_edge_maps.npy"

In [4]:
# Load the dataset
df = pd.read_csv(csv_path)
print(f"Dataset loaded: {len(df)} images")
print(f"Columns: {df.columns.tolist()}")
print(f"\nFirst few rows:")
print(df.head())

# Create dataset
dataset = UnifiedImageDataset(
    df.rename(columns={"local_path": "file_path"}), 
    mode="file_df",
    size=(224, 224)
)

print(f"\nDataset created with {len(dataset)} samples")

# Create data loader with optimal settings
num_cpu = cpu_count()
num_workers = min(32, max(8, num_cpu // 8))

loader = DataLoader(
    dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers,
    pin_memory=True,
    persistent_workers=True,
    prefetch_factor=6,
    pin_memory_device="cuda" if torch.cuda.is_available() else None,
)

print(f"\nDataLoader created with {len(loader)} batches")
print(f"Batch size: {batch_size}")
print(f"Number of workers: {num_workers}")


Dataset loaded: 2080 images
Columns: ['file_path', 'file_name', 'text', 'dimensions', 'width', 'height']

First few rows:
                                           file_path      file_name  \
0  /data/thesis/AdImageNet/images/(300, 250)/ad_0...  ad_000001.jpg   
1  /data/thesis/AdImageNet/images/(300, 250)/ad_0...  ad_000009.jpg   
2  /data/thesis/AdImageNet/images/(300, 250)/ad_0...  ad_000017.jpg   
3  /data/thesis/AdImageNet/images/(300, 600)/ad_0...  ad_000020.jpg   
4  /data/thesis/AdImageNet/images/(300, 250)/ad_0...  ad_000021.jpg   

                                                text  dimensions  width  \
0  $3\nSTULZ\nDifferential for 2nd Shift\nManufac...  (300, 250)    300   
1   VULTURE\ninto\nwith\nSam Sanders\nApple Podcasts  (300, 250)    300   
2  smart\ncare\nO\ndesign\nbuild\n& install\nrepa...  (300, 250)    300   
3  TREE\nSANTOR\nMatch On!\nThe Showstopper,\nLuc...  (300, 600)    300   
4  Local experts connecting\ncustomers to YOUR bu...  (300, 250)    300   



## 4. Generate Features and Train Model


In [5]:
# Generate CLIP embeddings using the package
print("Generating CLIP embeddings...")
embeddings = degis.generate_xl_embeddings(
    csv_path=csv_path,
    output_path=embeddings_path,
    batch_size=batch_size,
    num_workers=num_workers,
    force_recompute=False
)
print(f"✓ Generated embeddings with shape: {embeddings.shape}")

# Generate color histograms
print("\nGenerating color histograms...")
histograms = degis.generate_color_histograms(
    loader=loader,
    hist_path=colour_hist_path,
    hist_bins=8,
    force_recompute=False,
    color_space="hcl"
)
print(f"✓ Generated HCL histograms with shape: {histograms.shape}")

# Generate edge maps
print("\nGenerating edge maps...")
edge_maps = degis.generate_edge_maps(
    loader=loader,
    edge_maps_path=edge_maps_path,
    method="canny",
    force_recompute=False
)
print(f"✓ Generated edge maps with shape: {edge_maps.shape}")

# Train the color disentanglement model
print("\nTraining color disentanglement model...")
results = degis.train_color_model(
    embeddings_path=embeddings_path,
    histograms_path=colour_hist_path,
    hist_kind="hcl514",
    epochs=200,
    batch_size=4096,
    val_batch_size=8192,
    lr=1e-3,
    weight_decay=1e-2,
    blur=0.05,
    lambda_ortho=0.1,
    top_k=100,
    weighting=True,
)

print(f"\n✓ Training complete!")
print(f"Output directory: {results['output_dir']}")


Generating CLIP embeddings...
→ Loaded precomputed from /data/thesis/models/adimagenet_embeddings.npy (shape=(2080, 1024))
Saved: /data/thesis/models/adimagenet_embeddings.npy, shape: (2080, 1024)
✓ Generated embeddings with shape: (2080, 1024)

Generating color histograms...
### Color Histogram Generation [FAST, HCL] ###
Total images: 2080, Bins: 8, Dimensions: 514
Loaded from /data/thesis/data/adimagenet_color_histograms_hcl_514.npy
✓ Generated HCL histograms with shape: (2080, 514)

Generating edge maps...
[✓] Loaded cached edge maps from /data/thesis/data/adimagenet_edge_maps.npy
✓ Generated edge maps with shape: (2080, 50176)

Training color disentanglement model...
Run dir: /data/degis/runs/color_hcl514_tk100_b4096-20250906-103203
Rest dir: /data/degis/runs/rest_hcl514_tk100_b4096-20250906-103203
Loaded → emb: (2080, 1024) | hist: (2080, 514) | kind=hcl514


Epoch 01/200: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.42s/it, EMD=0.1126, T=1.00]


Epoch 01  train EMD=0.1126  val EMD=0.0755  (diag BCE=0.0141)
✓ saved best


Epoch 02/200: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.04it/s, EMD=0.1114, T=1.00]


Epoch 02  train EMD=0.1114  val EMD=0.0753  (diag BCE=0.0140)
✓ saved best


...

Epoch 13  train EMD=0.0695  val EMD=0.1452  (diag BCE=0.0201)


Epoch 14/200: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.16it/s, EMD=0.1032, T=1.00]


Epoch 14  train EMD=0.1032  val EMD=0.2104  (diag BCE=0.0229)


Epoch 15/200: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.21it/s, EMD=0.1352, T=1.00]


Epoch 15  train EMD=0.1352  val EMD=0.1046  (diag BCE=0.0253)
Early stop at epoch 15 (best val=0.0747)

✓ Training complete!
Output directory: /data/degis/runs/color_hcl514_tk100_b4096-20250906-103203
